In [1]:
import glob
import os
import tempfile
import warnings

import geopandas as gpd
from gplately.tools import plate_isotherm_depth

from lib.assign_regions import assign_regions
from lib.calculate_convergence import run_calculate_convergence
from lib.check_files import (
    check_source_data,
    check_plate_model,
)
from lib.combine_point_data import combine_point_data
from lib.coregister_combined_point_data import run_coregister_combined_point_data
from lib.coregister_crustal_thickness import run_coregister_crustal_thickness
from lib.coregister_magnetic import coregister_magnetic
from lib.coregister_ocean_rasters import run_coregister_ocean_rasters
from lib.create_study_area_polygons import run_create_study_area_polygons
from lib.erodep import calculate_erodep
from lib.generate_unlabelled_points import generate_unlabelled_points
from lib.misc import (
    calculate_slab_flux,
    calculate_water_thickness,
)
from lib.slab_dip import calculate_slab_dip

# Suppress occasional joblib warnings
%env PYTHONWARNINGS=ignore::UserWarning

warnings.simplefilter("ignore", UserWarning)


env: PYTHONWARNINGS=ignore::UserWarning


In [2]:
random_seed = 1234

n_jobs = int(os.environ.get("N_JOBS", 8))
times = range(171)
verbose = False

num_unlabelled = 200  # per timestep

os.environ['CURL_CA_BUNDLE'] = ''


### Input and output files

In [3]:
model_dir = "plate_model"
#check_plate_model(model_dir, verbose=verbose)

data_dir = "source_data"
#check_source_data(data_dir, verbose=verbose)
deposits_filename = os.path.join(data_dir, "deposit_data.csv")
regions_filename = os.path.join(data_dir, "regions.shp")
agegrid_dir = os.path.join(data_dir, "AgeGrids")
sedthick_dir = os.path.join(data_dir, "SedimentThickness")
carbonate_dir = os.path.join(data_dir, "CarbonateThickness")
co2_dir = os.path.join(data_dir, "CrustalCO2")
crustal_thickness_dir = os.path.join(data_dir, "CrustalThickness")
mag_anomaly_filename = os.path.join(
    data_dir,
    "MagneticAnomaly",
    "emag2_upcont_interpolated.nc",
)
subducted_quantities_dir = os.path.join(
    data_dir,
    "SubductedQuantities",
)
erodep_dir = os.path.join(data_dir, "ErosionDeposition")

output_dir = "extracted_data"
os.makedirs(output_dir, exist_ok=True)
subduction_data_filename = os.path.join(output_dir, "subducting_plate_data.csv")
study_area_dir = os.path.join(output_dir, "study_area_polygons")
output_filename = os.path.join(output_dir, "training_data.csv")

feature_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.gpml",
    )
)
rotation_filenames = glob.glob(
    os.path.join(
        model_dir,
        "*.rot",
    )
)
static_polygons_filename = os.path.join(
    model_dir,
    "StaticGeometries",
    "StaticPolygons",
    "Clennett_2020_StaticPolygons.gpml",
)


### Calculate cumulative erosion

In [4]:
#coregistered_data.to_csv('coregistered_data_mag.csv',index=False)
import pandas as pd
coregistered_data = pd.read_csv('coregistered_data_mag.csv')

In [5]:
verbose = True
coregistered_data = calculate_erodep(
    coregistered_data,
    input_dir=erodep_dir,
    n_jobs=n_jobs,
    column_name="erosion (m)",
    verbose=verbose,
)


banana
(37320, 57)
(37320,)
0 (389, 57)
Working on 0 Ma1 (224, 57)
Working on 1 Maextract erodep TIME: 1
source_data/ErosionDeposition
[]
ERODEP_TIMES: set()



ValueError: zero-size array to reduction operation maximum which has no identity

### Assign data to regions

In [ ]:
points = gpd.GeoSeries.from_xy(
    coregistered_data["present_lon"],
    coregistered_data["present_lat"],
    index=coregistered_data.index,
)
coregistered_data["region"] = assign_regions(
    points,
    regions=regions_filename,
)
del points


In [ ]:
coregistered_data.to_csv(output_filename, index=False)

coregistered_data.groupby(["source", "region", "label"]).size()


In [ ]:
#!conda list